# Install 

In [1]:
!pip uninstall gglitely -y
!pip install git+https://github.com/seoyeonc/gglitely.git

Found existing installation: gglitely 0.0.1
Uninstalling gglitely-0.0.1:
  Successfully uninstalled gglitely-0.0.1


  Cloning https://github.com/seoyeonc/gglitely.git to /tmp/pip-req-build-9t0e7z9w
  Running command git clone --filter=blob:none --quiet https://github.com/seoyeonc/gglitely.git /tmp/pip-req-build-9t0e7z9w


  Resolved https://github.com/seoyeonc/gglitely.git to commit 86f5c00a4a8e40a05e5fdaa927296f50ff290b24


  Preparing metadata (setup.py) ... -

 done


 \ done
  Created wheel for gglitely: filename=gglitely-0.0.1-py3-none-any.whl size=2473 sha256=0a6075f842842ec871a74e18a523d82c3160de8baa0a0ab9469016da09d2adf1
  Stored in directory: /tmp/pip-ephem-wheel-cache-zmt2m3x0/wheels/a5/b2/29/5db99646aa2b8d946b957bdf761a177612bf6709303d4ed579
Successfully built gglitely


# Imports 

In [2]:
from gglitely import *

In [3]:
subplot_count = lambda fig: len({geom[key] for geom in fig.data for key in geom if 'xaxis' in key})
def trace_axes_adjustment(fig1, fig2):
    x_axes_fig1 = sorted([key for key in fig1.layout if 'xaxis' in key])
    y_axes_fig1 = sorted([key for key in fig1.layout if 'yaxis' in key])
    x_axes_fig2 = sorted([key for key in fig2.layout if 'xaxis' in key])
    y_axes_fig2 = sorted([key for key in fig2.layout if 'yaxis' in key])    
    for trace in fig1.data:
        if trace['xaxis'] is None:
            for i, (x, y) in enumerate(zip(x_axes_fig1, y_axes_fig1), start=1):
                trace['xaxis'] = f'x{i}'
                trace['yaxis'] = f'y{i}'
    for trace in fig2.data:
        if trace['xaxis'] is None:
            for i, (x, y) in enumerate(zip(x_axes_fig2, y_axes_fig2), start=1):
                trace['xaxis'] = f'x{i + subplot_count(fig1)}'
                trace['yaxis'] = f'y{i + subplot_count(fig1)}'
        else:
            if trace['xaxis'] == 'x':
                trace['xaxis'] = f'x{1 + subplot_count(fig1)}'
            else:
                axis_num = int(trace['xaxis'][-1]) + subplot_count(fig1)
                trace['xaxis'] = f'x{axis_num}'

            if trace['yaxis'] == 'y':
                trace['yaxis'] = f'y{1 + subplot_count(fig1)}'
            else:
                axis_num = int(trace['yaxis'][-1]) + subplot_count(fig1)
                trace['yaxis'] = f'y{axis_num}'
    return fig1, fig2

def clayout_adjustment(fig1, fig2):
    # Loop through fig1 and fig2
    for f in [fig1, fig2]:
        # Extract all xaxes and yaxes
        xaxes = [geom[key] for geom in f.data for key in geom if 'xaxis' in key]
        yaxes = [geom[key] for geom in f.data for key in geom if 'yaxis' in key]
        # Check if all traces in the figure share the same xaxis (i.e., there is only one subplot)
        if len(set(xaxes)) == 1:
            for i, (x, y) in enumerate(zip(xaxes, yaxes), start=1):
                # Update xaxis layout
                f.layout[x.replace('x', 'xaxis')] = {'anchor': x.replace('axis', ''), 'domain': [0.0, 1.0]}
                # Update yaxis layout
                f.layout[y.replace('y', 'yaxis')] = {'anchor': y.replace('axis', ''), 'domain': [0.0, 1.0]}
        else:
            pass  # Do nothing if xaxes are not the same
    return fig1, fig2

    
def cbind(fig1,fig2,column_widths=None):
    # Update data and layout for fig1 and fig2
    fig1 = gglitely(data=fig1.data, layout=fig1.layout)
    fig2 = gglitely(data=fig2.data, layout=fig2.layout)
    
    # Calculate the number of subplots
    n1 = subplot_count(fig1)
    n2 = subplot_count(fig2)
    
    # Adjust trace axes
    fig1, fig2 = trace_axes_adjustment(fig1, fig2)
    
    # Adjust layout
    fig1, fig2 = clayout_adjustment(fig1, fig2)
    
    # Set column widths if not provided
    if column_widths is None:
        column_widths = [1] * n1 + [1] * n2
    
    # Create subplots
    fig = make_subplots(rows=1, cols=n1 + n2, column_widths=column_widths)
    fig = gglitely(layout=fig.layout)
    
    # Add traces from fig1 and fig2
    fig.add_traces(fig1.data)
    fig.add_traces(fig2.data)
    
    # Increment ncols
    fig._ncols += 1
    return fig

# 서브플랏 

In [4]:
fig1 = gglitely() + bar(y=[1,2,3],col='red') + bar(y=[2,3,1],col='blue')
fig2 = gglitely() + bar(y=[2,3,1],alpha=0.7,col='blue')

In [5]:
cbind(cbind(cbind(fig1,fig2),fig1),fig2)

AttributeError: 'gglitely' object has no attribute '_ncols'